# Data Science Capstone - Predicting Car Accident Severity
**Sergio Aguirre**
***

## **Introduction - Business Problem**

Daily we are faced with making decisions that, however simple they may seem, can change our lives. One of these decisions is which route to take to reach a specific destination.
Generally, the optimization and decision go through which route is shorter, will take less time or is less congested, but what about security?.

The goal of this work is to analyze data in order to provide a model that will predict the severity of an accident, given a number of features that will be extracted from the car collision data from Seattle. The model will classify and predict severity, so a driver can use this model through a third-party or government app to change course or drive more carefully, given that he is not wanting to get involved into an accident.

This project will use Machine Learning to build models to predict the severity of the car accidents according to the external factors in the car collision data of the city of Seattle. This model can be applied to multiple practical conditions, such as route planning, signals placement or road improvements. This model can be methodologically extrapolated to other cities anywhere in the world that have similar data collection systems.

This project is aimed at government agencies and private companies that want to optimize their route planning and prevent accidents.

## Data

The car collision data that will be used comes from the Seattle Department of Transportation, which records the information related to traffic accidents and the associated factors. This dataset includes a severity indicator (SEVERITYCODE) for each traffic accident where the values are 1 for minors and 2 for severe accidents. 
It also includes external information associated with accidents that is of direct utility for the study, such as weather, road and lighting conditions. Obviously crucial information about the date and time of occurrence, as well as internal factors such as the influence of alcohol (UNDERINFL), speeding (SPEEDING) or inattention that allow us to rule out certain accidents from our data that do not have to do purely with external factors. For a complete description of each parameter, consult the metadata included in the repository.

The data looks like the following:

![alt text](External_Condition.png "Logo Title Text 1")